In [111]:
import pandas as pd
import numpy as np
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.metrics import confusion_matrix as cm
from sklearn.metrics import classification_report as cr
# names=
train = pd.read_csv("Census_Income.csv",header=None)
train.columns=['age','workclass','fnlwgt','education','education_num','marital_status','occupation','relationship','race','sex','capital_gain','capital_loss','hours_per_week','native_country','salary']
# workclass ? to Private, drop education column as it is same as education-num, divide occupation ? 30% to Prof-specialty,Craft-repair,Exec-managerial, drop capital gain/loss as >29000 is 0, native country - ? with United-States
train.workclass=train.workclass.str.replace("?","Private")
# train["workclass"].value_counts()
train.native_country=train.native_country.str.replace("?","United-States")
# train["native_country"].value_counts()
train.occupation=train.occupation.str.replace("?","Prof-specialty")
train["occupation"].value_counts()
train=train.drop(columns=["education","capital_gain","capital_loss"])
# train.head()
# encodee: workclass, marital-status, occupation, relationship, race, sex, native-country, salary
# for col in train:
  # print(col)
  # print(len(train[col].unique()))
  # print(train[col].value_counts())
nulCount=train.isna().sum().sum()
# print(nulCount)
if nulCount>0:
  for col in train:
    count=train[col].isna().sum()
    if count>0:
      if(is_numeric_dtype(train[col])):
        avg=train[col].mean(skipna=True)
        train[col]=train[col].fillna(avg)
      elif(is_string_dtype(train[col])):
        max=train[col].value_counts().idxmax()
        train[col]=train[col].fillna(max)

# print(train.describe(include="all"))
# print(train)
# print(train.isna().sum())
le=preprocessing.LabelEncoder()
# train["salary"]=np.where(train["salary"]=='>50K',0,1)
train.salary=train.salary.str.replace("<=50K","0")
train.salary=train["salary"].str.replace(">50K","1")
print(train["salary"].value_counts())
for col in ['workclass','marital_status','occupation','relationship','race','sex','native_country']:
  train[col]=le.fit_transform(train[col])
y=train["salary"]
X=train.drop(columns="salary")
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=4)
# print(X_train.shape)
# y_train
rfc=rf(n_estimators=100, max_depth=10)
rfc=rfc.fit(X_train,y_train)
pred=rfc.predict(X_test)
pred
print(cm(y_test,pred))
print(cr(y_test, pred))

 0    24720
 1     7841
Name: salary, dtype: int64
[[5715  457]
 [ 907 1062]]
              precision    recall  f1-score   support

           0       0.86      0.93      0.89      6172
           1       0.70      0.54      0.61      1969

    accuracy                           0.83      8141
   macro avg       0.78      0.73      0.75      8141
weighted avg       0.82      0.83      0.82      8141

